In [46]:
def safe_version(pkg):
    try:
        import importlib.metadata as im
        return im.version(pkg)
    except Exception:
        return "no instalado"

print("pyspark:", safe_version("pyspark"))
print("delta-spark:", safe_version("delta-spark"))
print("py4j:", safe_version("py4j"))

pyspark: 3.5.1
delta-spark: 3.2.0
py4j: 0.10.9.7


In [47]:
from pyspark.sql import SparkSession

try:
    # Probar si Spark arranca
    spark = (SparkSession.builder
             .appName("spark-smoke-test")
             .getOrCreate())
    print("Spark arrancó ✅")
    print("Spark (spark.version):", spark.version)
    # versión del backend (a veces útil):
    try:
        print("Spark (jvm _jsc.version()):", spark.sparkContext._jsc.version())
    except Exception:
        pass
    spark.range(1).show()
finally:
    try:
        spark.stop()
    except Exception:
        pass

Spark arrancó ✅
Spark (spark.version): 3.5.1
Spark (jvm _jsc.version()): 3.5.1
+---+
| id|
+---+
|  0|
+---+



In [48]:
# Crear builder
from pyspark.sql import SparkSession

DELTA_VERSION = "3.2.0"  # compatible con Spark 3.5.x (Scala 2.12)

spark = (
    SparkSession.builder
    .appName("EDA_Calidad_Landing")
    # Extensiones Delta
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    # 👉 Añadir los JAR de Delta desde Maven Central
    .config("spark.jars.packages", f"io.delta:delta-spark_2.12:{DELTA_VERSION}")
    .config("spark.sql.shuffle.partitions", "200")
    .getOrCreate()
)

print("Spark:", spark.version)
print("Extensiones:", spark.conf.get("spark.sql.extensions"))
print("Packages:", spark.sparkContext.getConf().get("spark.jars.packages"))

Spark: 3.5.1
Extensiones: io.delta.sql.DeltaSparkSessionExtension
Packages: io.delta:delta-spark_2.12:3.2.0


In [49]:
from pathlib import Path

# Root del proyecto: subir dos niveles desde notebooks/01_data_analytics/
PROJECT_ROOT = Path().resolve().parents[1]
LANDING_PATH = PROJECT_ROOT / "data" / "landing" / "aemet_deltalake"

try:
    df = spark.read.format("delta").load(str(LANDING_PATH))   # si hay _delta_log
except Exception:
    df = spark.read.parquet(str(LANDING_PATH / "*.parquet"))  # fallback parquet

print("Filas:", df.count(), "| Columnas:", len(df.columns))
df.printSchema()

Filas: 2682447 | Columnas: 26
root
 |-- fecha: string (nullable = true)
 |-- indicativo: string (nullable = true)
 |-- nombre: string (nullable = true)
 |-- provincia: string (nullable = true)
 |-- altitud: string (nullable = true)
 |-- tmed: string (nullable = true)
 |-- prec: string (nullable = true)
 |-- tmin: string (nullable = true)
 |-- horatmin: string (nullable = true)
 |-- tmax: string (nullable = true)
 |-- horatmax: string (nullable = true)
 |-- hrMax: string (nullable = true)
 |-- horaHrMax: string (nullable = true)
 |-- hrMin: string (nullable = true)
 |-- horaHrMin: string (nullable = true)
 |-- hrMedia: string (nullable = true)
 |-- dir: string (nullable = true)
 |-- velmedia: string (nullable = true)
 |-- racha: string (nullable = true)
 |-- horaracha: string (nullable = true)
 |-- presMax: string (nullable = true)
 |-- horaPresMax: string (nullable = true)
 |-- presMin: string (nullable = true)
 |-- horaPresMin: string (nullable = true)
 |-- sol: string (nullable = tru

# Completitud del dataset y distribución

Checks incluidos:
1) Número total de registros del dataset
2) **Duplicados** en la clave (`fecha`, `indicativo`) — no debería haber
3) **Registros por estación** (comparación con esperado = 365 × años 2017–2025)
   - Estaciones con **menos de lo esperado**
   - Valor medio entre todas las estaciones
   - Estaciones con **más de lo normal**
4) **Registros por estación y año** (para estaciones con menos registros totales)
   - Distribución por año para detectar si faltan sistemáticamente en mismas fechas o en años concretos

**Tecnologías**: Spark para agregados, Altair para visualización interactiva.

In [39]:
from pyspark.sql.functions import col, to_date, year, count, lit, min as Fmin, max as Fmax, avg, stddev
from datetime import date

# Asegurar que 'fecha' es de tipo DATE
if "fecha" in df.columns:
    df = df.withColumn("fecha", to_date(col("fecha")))

# Parámetros del análisis (rango real del dataset)
START_DATE = date(2017, 1, 1)
END_DATE   = date(2025, 6, 30)

# Máximo de registros esperados por estación = días entre START_DATE y END_DATE (incluidos)
EXPECTED_MAX_PER_STATION = (END_DATE - START_DATE).days + 1

print(f"Rango analizado: {START_DATE} → {END_DATE}")
print("Registros esperados por estación (máximo exacto):", EXPECTED_MAX_PER_STATION)

Rango analizado: 2017-01-01 → 2025-06-30
Registros esperados por estación (máximo exacto): 3103


In [40]:
# Numero de obsrvaciones diarias
df.groupBy("fecha").agg(count(lit(1)).alias("num_registros")).orderBy("fecha").show(10, truncate=False)

+----------+-------------+
|fecha     |num_registros|
+----------+-------------+
|2017-01-01|849          |
|2017-01-02|849          |
|2017-01-03|850          |
|2017-01-04|852          |
|2017-01-05|850          |
|2017-01-06|848          |
|2017-01-07|849          |
|2017-01-08|849          |
|2017-01-09|848          |
|2017-01-10|848          |
+----------+-------------+
only showing top 10 rows



## 1) Número de registros totales

In [50]:
total_registros = df.count()
print("Total de registros en el dataset:", f"{total_registros:,}")

# Total estaciones
total_estaciones = df.select("indicativo").distinct().count()
print("Total de estaciones:", f"{total_estaciones:,}")

# Total esperado
expected_total = total_estaciones * EXPECTED_MAX_PER_STATION
print("Total esperado (estaciones × registros por estación):", f"{expected_total:,}")

Total de registros en el dataset: 2,682,447
Total de estaciones: 918
Total esperado (estaciones × registros por estación): 2,848,554


In [ ]:
%%sql


Faltan registros para completar el total esperado.

## 2) Duplicados en la clave (fecha, indicativo)
No debería haber duplicados por combinación `fecha` + `indicativo`.
- Contamos duplicados globales.
- Contamos duplicados por año para ver si hay algún patrón en años concretos.

In [51]:
# Duplicados globales
dups_global = (df.groupBy("fecha","indicativo").count().filter(col("count") > 1)).count()
print("Duplicados totales (clave fecha+indicativo):", dups_global)

# Duplicados por año
df_dups_year = (df.groupBy("fecha","indicativo")
                  .count()
                  .filter(col("count") > 1)
                  .withColumn("año", year(col("fecha")))
                  .groupBy("año")
                  .agg(count(lit(1)).alias("duplicados"))
                  .orderBy("año"))
df_dups_year.show()

Duplicados totales (clave fecha+indicativo): 0


25/08/21 18:35:50 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+---+----------+
|año|duplicados|
+---+----------+
+---+----------+



No hay duplicados en la clave `fecha` + `indicativo`.

## 3) Número de registros por estación
- Comparamos con el esperado: `365 × años (2017–2025) = 3103`.
- Calculamos media y desviación estándar.
- Identificamos estaciones con **menos** de lo esperado y con **más** de lo normal (> `3287` si permitimos bisiestos).

In [52]:
# Agregado por estación
df_by_station = df.groupBy("indicativo").agg(count(lit(1)).alias("registros"))
df_by_station = df_by_station.orderBy(col("registros").desc())

# Estadísticos globales sobre estaciones
station_stats = df_by_station.agg(
    avg("registros").alias("media_registros"),
    stddev("registros").alias("stddev_registros")
).collect()[0]
print("Media de registros por estación:", round(station_stats["media_registros"], 2))
print("Desv. estándar:", round(station_stats["stddev_registros"] or 0.0, 2))


Media de registros por estación: 2922.06
Desv. estándar: 497.6


# TO DO
Ampliar estadistica sobre estaciones

# Visualizar estaciones segun su cobertura

In [53]:
df_by_station.show(10, truncate=False)

# Numero maximo de registros por estacion
from pyspark.sql.functions import max as Fmax

max_registros = df_by_station.agg(Fmax("registros").alias("max_registros")).collect()[0]["max_registros"]
print("Máximo de registros en una estación:", max_registros)
max_row = df_by_station.orderBy(col("registros").desc()).first()

# Numero de estaciones con numero maximo de registros
max_stations = df_by_station.filter(col("registros") == max_registros).count()
print(f"Estaciones con {max_registros} registros: {max_stations}")


# Estacion con menos registros (minimo)
N = 10
print(f"Lista de estaciones con menos registros")
df_by_station.orderBy(col("registros").asc()).show(N, truncate=False)


+----------+---------+
|indicativo|registros|
+----------+---------+
|1276F     |3102     |
|7158X     |3102     |
|1010X     |3102     |
|9031C     |3102     |
|C129V     |3102     |
|C929I     |3102     |
|C430E     |3102     |
|1387E     |3102     |
|4386B     |3102     |
|3434X     |3102     |
+----------+---------+
only showing top 10 rows

Máximo de registros en una estación: 3102
Estaciones con 3102 registros: 173
Lista de estaciones con menos registros
+----------+---------+
|indicativo|registros|
+----------+---------+
|9283X     |43       |
|1437O     |208      |
|2661B     |212      |
|5038X     |326      |
|6332X     |344      |
|2503X     |344      |
|6299I     |385      |
|9576C     |521      |
|5511      |581      |
|6248D     |623      |
+----------+---------+
only showing top 10 rows



In [54]:
# Ver distribución de estaciones por % de cobertura

from pyspark.sql.functions import count, lit, col
import pandas as pd
import numpy as np
import altair as alt
from IPython.display import display
from datetime import date

# 1) Asegura df_by_station
try:
    df_by_station
except NameError:
    df_by_station = (df.groupBy("indicativo")
                       .agg(count(lit(1)).alias("registros")))

# 2) Esperado exacto (2017-01-01 a 2025-06-30)
EXPECTED_MAX_PER_STATION = (date(2025,6,30) - date(2017,1,1)).days + 1  # 3110

# 3) A Pandas y cobertura %
station_pd = df_by_station.toPandas()
station_pd["coverage_pct"] = (station_pd["registros"] / EXPECTED_MAX_PER_STATION) * 100

# 4) Limpiar valores no válidos
before = len(station_pd)
station_pd = station_pd.replace([np.inf, -np.inf], np.nan).dropna(subset=["coverage_pct"])
station_pd = station_pd[(station_pd["coverage_pct"] >= 0) & (station_pd["coverage_pct"] <= 100)]
after = len(station_pd)
if after == 0:
    raise ValueError("No hay datos válidos en 'coverage_pct' tras la limpieza. Revisa 'registros' y el esperado.")

print(f"Estaciones válidas para gráfico: {after}/{before}")

# 5) Render Altair para PyCharm
try:
    alt.renderers.enable('html')
except Exception:
    pass

# 6) Binning y % con Altair (sin columnas null)
N = len(station_pd)

chart = (
    alt.Chart(station_pd)
    .transform_bin("bin_cov", field="coverage_pct", bin=alt.Bin(extent=[0, 100], step=10))
    .transform_aggregate(n_estaciones="count()", groupby=["bin_cov", "bin_cov_end"])
    .transform_calculate(
        pct_estaciones=f"(datum.n_estaciones / {N}) * 100",
        bin_label="format(datum.bin_cov, '.0f') + '–' + format(datum.bin_cov_end, '.0f')"
    )
    .mark_bar()
    .encode(
        x=alt.X("bin_label:N", title="Cobertura de registros (%)", sort=None),
        y=alt.Y("pct_estaciones:Q", title="% de estaciones"),
        tooltip=["bin_label:N","n_estaciones:Q","pct_estaciones:Q"]
    )
    .properties(title="Distribución de estaciones por % de cobertura", width=650, height=300)
)

# 7) Etiquetas  encima de las barras
labels = chart.mark_text(dy=-5).encode(text=alt.Text("pct_estaciones:Q", format=".1f"))

display(chart + labels)

Estaciones válidas para gráfico: 918/918


alt.LayerChart(...)

1. Cobertura alta en la mayoría de estaciones (>90%)
- En total, ~89,5% de las estaciones tienen más del 90% de los días cubiertos.

2. Cobertura media (80–90%)
- Un grupo pequeño (~2,6%) se reparte entre los tramos 80–95%.

3. Cobertura baja (<80%)
- ~8% (7,9%) de estaciones tienen menos del 80% de cobertura, y unas pocas incluso <20%

## Garantizar que la cobertura alta tiene igual comportamiento

Ver distribución de estaciones con cobertura alta (>90%), media (80-90%) y baja (<80%).

In [34]:
from pyspark.sql.functions import col, count, lit
from datetime import date

from pathlib import Path

PROJECT_ROOT = Path().resolve().parents[1]
LANDING_PATH = PROJECT_ROOT / "data" / "landing" / "aemet"
LANDING_PATH_STATIONS = LANDING_PATH / "aemet_stations.parquet"

stations_df = spark.read.parquet(str(LANDING_PATH_STATIONS))
stations_pd = stations_df.select("indicativo", "nombre", "provincia", "latitud", "longitud").toPandas()


# --- 2) Función para convertir '394924N' / '0031530W' a decimal ---
def dms_compact_to_decimal(s):
    if pd.isna(s):
        return np.nan
    s = str(s).strip().upper()
    if len(s) < 2:
        return np.nan
    hemi = s[-1]  # N/S/E/W
    digits = s[:-1]
    try:
        if hemi in ("N","S"):  # latitud: DDMMSS
            d = int(digits[0:2]); m = int(digits[2:4]); sec = int(digits[4:6])
        else:                  # longitud: DDDMMSS
            d = int(digits[0:3]); m = int(digits[3:5]); sec = int(digits[5:7])
        val = d + m/60.0 + sec/3600.0
        if hemi in ("S","W"):
            val = -val
        return val
    except Exception:
        return np.nan

# Convertir coordenadas
stations_pd["lat"] = stations_pd["latitud"].apply(dms_compact_to_decimal)
stations_pd["lon"] = stations_pd["longitud"].apply(dms_compact_to_decimal)

# Limpiar filas sin coordenadas válidas
stations_pd = stations_pd.dropna(subset=["lat","lon"])

# --- 3) Cobertura por estación (df_by_station debe existir de antes) ---
# Si no existe, lo calculamos:
try:
    df_by_station
except NameError:
    df_by_station = (df.groupBy("indicativo").agg(count(lit(1)).alias("registros")))

coverage_pd = df_by_station.toPandas()

# Rango real analizado: 2017-01-01 a 2025-06-30 => 3110 días
EXPECTED_MAX_PER_STATION = (date(2025,6,30) - date(2017,1,1)).days + 1  # 3110

coverage_pd["coverage_pct"] = (coverage_pd["registros"] / EXPECTED_MAX_PER_STATION) * 100

def classify(p):
    if p >= 95:
        return "Alta (≥90%)"
    elif p >= 80:
        return "Media (80–90%)"
    else:
        return "Baja (<80%)"

coverage_pd["cobertura"] = coverage_pd["coverage_pct"].apply(classify)

# --- 4) Unión estaciones + cobertura ---
merged_pd = coverage_pd.merge(stations_pd, on="indicativo", how="left")
print("Estaciones totales unidas:", len(merged_pd))
merged_pd.head()

Estaciones totales unidas: 918


,indicativo,registros,coverage_pct,cobertura,nombre,provincia,latitud,longitud,lat,lon
0,9808X,3092,99.645504,Alta (≥90%),"AÍNSA-SOBRARBE, LA SERRETA",HUESCA,422518N,000817E,42.421667,1.351944
1,4347X,3088,99.516597,Alta (≥90%),ZORITA,CACERES,391658N,054237W,39.282778,-54.385278
2,1159,3091,99.613277,Alta (≥90%),SAN VICENTE DE LA BARQUERA,CANTABRIA,432336N,042332W,43.393333,-42.550556
3,9060X,3047,98.195295,Alta (≥90%),LALASTRA,ARABA/ALAVA,425231N,031354W,42.875278,-31.584444
4,9563X,3083,99.355462,Alta (≥90%),CASTELLFORT,CASTELLON,402955N,001112W,40.498611,-1.183889


In [32]:
import re
import numpy as np
import pandas as pd
from datetime import date
from pyspark.sql.functions import col, count, lit
import folium

# --- 1) Leer inventario de estaciones ---
from pathlib import Path

PROJECT_ROOT = Path().resolve().parents[1]
LANDING_PATH = PROJECT_ROOT / "data" / "landing" / "aemet"
LANDING_PATH_STATIONS = LANDING_PATH / "aemet_stations.parquet"

stations_df = spark.read.parquet(str(LANDING_PATH_STATIONS))
stations_pd = stations_df.select("indicativo", "nombre", "provincia", "latitud", "longitud").toPandas()

# --- 2) Función robusta DMS compacto -> decimal ---
# Acepta formatos tipo: 394924N, 394924n, 024224E, 0031530W, con o sin ceros a la izquierda.
def dms_compact_to_decimal(s, is_lat=None):
    if pd.isna(s):
        return np.nan
    s = str(s).strip().upper()
    # Extraer hemisferio (última letra si pertenece a N/S/E/W)
    m = re.match(r"^(\d+)\s*([NSEW])$", s)
    if not m:
        return np.nan
    digits, hemi = m.group(1), m.group(2)
    # Si no indican si es lat o lon, inferir por longitud de dígitos (lat ~6, lon ~7), pero sé tolerante.
    if is_lat is None:
        is_lat = (len(digits) <= 6)  # 6: DDMMSS ; 7: DDDMMSS

    # Pad con ceros para asegurar longitud esperada
    if is_lat:
        digits = digits.zfill(6)  # DDMMSS
        d = int(digits[0:2])
        m_ = int(digits[2:4])
        s_ = int(digits[4:6])
    else:
        digits = digits.zfill(7)  # DDDMMSS
        d = int(digits[0:3])
        m_ = int(digits[3:5])
        s_ = int(digits[5:7])

    val = d + m_/60.0 + s_/3600.0
    if hemi in ("S","W"):
        val = -val
    # Validación global
    if is_lat and not (-90 <= val <= 90):
        return np.nan
    if (not is_lat) and not (-180 <= val <= 180):
        return np.nan
    return val

# Convertir coordenadas
stations_pd["lat"] = stations_pd["latitud"].apply(lambda x: dms_compact_to_decimal(x, is_lat=True))
stations_pd["lon"] = stations_pd["longitud"].apply(lambda x: dms_compact_to_decimal(x, is_lat=False))

# --- 3) Filtrar coordenadas inválidas ---
before = len(stations_pd)
stations_pd = stations_pd.dropna(subset=["lat","lon"])
invalid_dropped = before - len(stations_pd)

# --- 4) Filtro geográfico: España (incluye Canarias y Baleares)
# Caja amplia para no cortar islas: lat ~ [27, 44.5], lon ~ [-19.5, 5.0]
SPAIN_LAT_MIN, SPAIN_LAT_MAX = 27.0, 44.5
SPAIN_LON_MIN, SPAIN_LON_MAX = -19.5, 5.0

in_es = (
    (stations_pd["lat"] >= SPAIN_LAT_MIN) & (stations_pd["lat"] <= SPAIN_LAT_MAX) &
    (stations_pd["lon"] >= SPAIN_LON_MIN) & (stations_pd["lon"] <= SPAIN_LON_MAX)
)
oob_dropped = (~in_es).sum()
stations_pd = stations_pd[in_es].copy()

print(f"Eliminadas por coordenadas inválidas: {invalid_dropped}")
print(f"Eliminadas por fuera de España (bbox): {oob_dropped}")
print(f"Estaciones válidas para mapa: {len(stations_pd)}")

# --- 5) Cobertura por estación (si df_by_station no existe, calcular) ---
try:
    df_by_station
except NameError:
    df_by_station = (df.groupBy("indicativo").agg(count(lit(1)).alias("registros")))

coverage_pd = df_by_station.toPandas()

# Rango real analizado: 2017-01-01 a 2025-06-30 => 3110 días
EXPECTED_MAX_PER_STATION = (date(2025,6,30) - date(2017,1,1)).days + 1  # 3110
coverage_pd["coverage_pct"] = (coverage_pd["registros"] / EXPECTED_MAX_PER_STATION) * 100

def classify(p):
    if p >= 95:   return "Alta (≥95%)"
    if p >= 80:   return "Media (80–95%)"
    return "Baja (<80%)"

coverage_pd["cobertura"] = coverage_pd["coverage_pct"].apply(classify)

# --- 6) Unión estaciones + cobertura ---
merged_pd = coverage_pd.merge(stations_pd, on="indicativo", how="inner")

# --- 7) Mapa folium ---
m = folium.Map(location=[40.3, -3.7], zoom_start=5, tiles="cartodbpositron")
def color_for(group):
    return {"Alta (≥95%)":"#1a9850", "Media (80–95%)":"#fee08b", "Baja (<80%)":"#d73027"}.get(group, "gray")

for _, r in merged_pd.iterrows():
    folium.CircleMarker(
        location=[r["lat"], r["lon"]],
        radius=5,
        color=color_for(r["cobertura"]),
        fill=True, fill_opacity=0.9,
        popup=folium.Popup(
            f"<b>{r.get('nombre','')}</b><br>"
            f"Indicativo: {r.get('indicativo','')}<br>"
            f"Provincia: {r.get('provincia','')}<br>"
            f"Cobertura: {r.get('coverage_pct',0):.1f}% "
            f"({int(r.get('registros',0))} / {EXPECTED_MAX_PER_STATION})",
            max_width=260
        )
    ).add_to(m)

m  # se muestra inline en PyCharm/Notebook
# m.save("mapa_cobertura_estaciones.html")  # opcional: guardar

Eliminadas por coordenadas inválidas: 0
Eliminadas por fuera de España (bbox): 0
Estaciones válidas para mapa: 947


Las estaciones con cobertura baja estan repartifas uniformemente por España, sin concentraciones en zonas concretas.


# TO DO
De las estaciones con cobertura media o baja, se puede:

- *Verificar si faltan datos de años concretos* (si es un patrón sistemático).
- *Verificar si faltan datos de fechas concretas*